In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import os


import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../input/2012_qb_weeks1_8.csv')
df1 = pd.read_csv('../input/2012_qb_weeks_9_16.csv')
df3 = pd.read_csv('../input/2012_qb_week17.csv')


In [3]:
season_schedule = pd.read_csv('../input/season_schedule1.csv')


In [137]:
season_schedule.head()

,Week,Day,Date,Time,away_team,home_team,game_time
0,1,Wed,5-Sep,8:41PM,Dallas Cowboys,New York Giants,"5-Sep,8:41PM"
1,1,Sun,9-Sep,1:02PM,Indianapolis Colts,Chicago Bears,"9-Sep,1:02PM"
2,1,Sun,9-Sep,1:03PM,Buffalo Bills,New York Jets,"9-Sep,1:03PM"
3,1,Sun,9-Sep,1:03PM,Philadelphia Eagles,Cleveland Browns,"9-Sep,1:03PM"
4,1,Sun,9-Sep,1:03PM,St. Louis Rams,Detroit Lions,"9-Sep,1:03PM"


In [107]:
season_schedule.drop('Winner/tie', axis =1, inplace =True)
season_schedule.drop('Loser/tie', axis =1, inplace =True)

In [129]:
season_schedule['game_time'] = season_schedule[season_schedule.columns[2:4]].apply(lambda x: ','.join(x.astype(str)), axis=1)

In [99]:
home_team = []
away_team = []
for game in season_schedule.values:
    if '@' in game[5]:
        home_team.append(game[6])
        away_team.append(game[4])
    else:
        home_team.append(game[4])
        away_team.append(game[6])

In [145]:
pd.to_datetime(season_schedule['game_time'][0]


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-09-05 20:41:00

In [141]:
home_team = []
away_team = []
for game in season_schedule.values:
    if '@' in game[5]:
        home_team.append(game[6])
        away_team.append(game[4])
    else:
        home_team.append(game[4])
        away_team.append(game[6])

season_schedule['away_team'] = away_team
season_schedule['home_team'] = home_team

TypeError: argument of type 'float' is not iterable

In [4]:
raw_qb_2012 = pd.concat([df,df1], axis=0)
raw_qb_2012 = pd.concat([raw_qb_2012, df3], axis=0)
raw_qb_2012.drop('Rk', axis =1, inplace = True)
raw_qb_2012.shape

(629, 18)

In [5]:
raw_qb_2012.Team.unique()

array(['ATL', 'WAS', 'DAL', 'GB', 'NO', 'NYJ', 'DEN', 'BAL', 'JAX', 'CHI',
       'OAK', 'SF', 'NE', 'KC', 'PIT', 'PHI', 'BUF', 'HOU', 'LAC', 'NYG',
       'CAR', 'DET', 'LAR', 'TB', 'TEN', 'IND', 'MIN', 'SEA', 'ARI', 'CIN',
       'MIA', 'CLE'], dtype=object)

In [29]:
url=('https://www.pro-football-reference.com/years/2012/games.htm')
html = urlopen(url)
bsobj= BeautifulSoup(html, 'lxml')

1072

In [28]:
bsobj.tbody.find_all('td', class_ = 'left')

NameError: name 'bsobj' is not defined

In [6]:
raw_qb_2012 = raw_qb_2012.rename(columns={'Att.1':'rush_att',
                       'Yds.1':'rush_yds',
                       'Yds/Att.1':'rush_yd/att',
                       'TD.1':'rush_td'})

In [13]:
team_stats = raw_qb_2012.groupby(['Player','Team','Week','Opp']).sum()

In [22]:
team_stats.index.levels[2]

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64', name='Week')

In [9]:
# Define a dataframe to contain the current year averages of all 2012 QBs
def create_current_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('avg_{}'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [15]:
qb_current_2012= create_current_dataframe(team_stats)

In [28]:
# Populate current dataframe with current year averages of players
# DO NOT CHANGE!!!!!!
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    points=[]  # Create an empty list to hold the season average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total =[] # create an empty list to hold the weekly attributes value
        count = 0
        for week in team_stats.loc[player].index: 
            if count == 0:     # Iterate throough every week for the current player
                points.append(0)
                total.append(team_stats.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                count += 1
            else:
                total.append(team_stats.loc[player].loc[week][col])
                points.append(np.mean(total))  # Add the average of the total list to the season average list
    qb_current_2012['avg_{}'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    

In [26]:
qb_current_2012.shape

(629, 13)

In [30]:
qb_current_2012[0:100]

avg_comp    avg_att    avg_pct     avg_yds  \
Player             Team Week Opp                                                
Aaron Rodgers      GB   1    SF    0.000000   0.000000   0.000000    0.000000   
                        2    CHI  26.000000  38.000000  68.500000  261.000000   
                        3    SEA  26.000000  38.333333  68.000000  248.333333   
                        4    NO   27.250000  39.000000  70.000000  266.000000   
                        5    IND  25.800000  37.600000  68.600000  259.800000   
                        6    HOU  25.500000  37.500000  68.000000  272.833333   
                        7    LAR  26.142857  37.428571  69.857143  282.714286   
                        8    JAX  25.625000  37.125000  69.000000  270.625000   
                        9    ARI  24.333333  36.333333  66.555556  264.777778   
                        11   DET  23.800000  35.400000  66.900000  261.900000   
                        12   NYG  22.909091  34.454545  65.909091  258.000000   
                        13   MIN  23.250000  34.500000  66.833333  260.333333   
                        14   DET  22.538462  33.692308  66.153846  253.615385   
                        15   CHI  22.571429  33.857143  66.000000  256.285714   
                        16   TEN  22.866667  34.133333  66.333333  262.000000   
                        17   MIN  23.187500  34.500000  66.562500  268.437500   
Alex Smith         SF   1    GB    0.000000   0.000000   0.000000    0.000000   
                        2    DET  20.000000  28.500000  71.000000  218.500000   
                        3    MIN  21.333333  30.666667  70.333333  213.666667   
                        4    NYJ  19.000000  28.250000  67.000000  196.000000   
                        5    BUF  18.800000  27.400000  68.600000  217.400000   
                        6    NYG  18.833333  27.833333  67.666667  214.500000   
                        7    SEA  18.142857  27.142857  66.714286  203.857143   
                        8    ARI  18.125000  26.125000  70.250000  207.375000   
                        10   LAR  16.888889  24.111111  72.222222  192.333333   
                        11   CHI  15.200000  21.700000  65.000000  173.100000   
                        12   NO   13.818182  19.727273  59.090909  157.363636   
                        13   LAR  12.666667  18.083333  54.166667  144.250000   
                        14   MIA  11.692308  16.692308  50.000000  133.153846   
                        15   NE   10.857143  15.500000  46.428571  123.642857   
...                                     ...        ...        ...         ...   
Ben Roethlisberger PIT  8    WAS  25.571429  38.285714  67.142857  283.857143   
                        9    NYG  25.000000  37.250000  67.500000  275.375000   
                        10   KC   23.222222  35.111111  65.555556  254.111111   
                        14   LAC  23.100000  35.800000  64.200000  257.200000   
                        15   DAL  23.181818  36.181818  63.818182  264.636364   
                        16   CIN  22.416667  35.500000  62.666667  260.916667   
                        17   CLE  21.846154  34.538462  62.846154  251.153846   
Blaine Gabbert     JAX  1    MIN   0.000000   0.000000   0.000000    0.000000   
                        2    HOU  15.000000  29.000000  48.000000  156.500000   
                        3    IND  13.333333  26.333333  48.000000  156.000000   
                        4    CIN  15.750000  28.250000  53.000000  163.500000   
                        5    CHI  16.000000  29.200000  52.800000  159.200000   
                        7    OAK  14.666667  26.333333  55.166667  151.000000   
                        8    GB   16.428571  29.571429  55.142857  172.714286   
                        9    DET  17.750000  30.625000  57.125000  178.625000   
                        10   IND  17.777778  30.666667  57.222222  182.000000   
                        11   HOU  16.200000  27.800000  61.500000 

In [52]:
qb_current_2012['date']

ValueError: Length of values does not match length of index